# End to end example

In [1]:
! pwd
import sys
sys.path.append('/Users/niarfe/tmprepos/hydra_inc/theseus_node')
sys.path.append('/Users/niarfe/tmprepos/hydra_inc/theseus_node/tests')
sys.path.append('/Users/niarfe/tmprepos/hydra_inc/hydraseq')

import theseus
import os
import data
import hydraseq as hdh

/Users/niarfe/tmprepos/hydra_inc/theseus_node/notebooks


In [2]:
! ls ../tests/data/iris
! cd ../tests/data/iris; wc -l iris_data.csv
seto, virg, vers = [], [], []
full = []
hydrav = []
with open('../tests/data/iris/iris_data.csv', 'r') as source:
    for line in source:
        arr = {
            'Iris-setosa':     seto,
            'Iris-versicolor': vers,
            'Iris-virginica':  virg
        }[line.split(',')[-1].strip()]
        arr.append([f+m for m, f in zip(line.split(',')[:-1], ['a','b','c','d'])])
        full.append([f+m for m, f in zip(line.split(',')[:-1], ['a','b','c','d'])])
        hydrav.append([e.strip() for e in line.split(',')])

print('done')

iris_data.csv
     150 iris_data.csv


KeyError: 'Species'

In [ ]:
n_seto = theseus.Node(seto)
n_virg = theseus.Node(virg)
n_vers = theseus.Node(vers)
n_back = theseus.Node(seto + vers + virg)
print('done')

In [ ]:
n_seto.visualize(n_back, cutoff=100, magnification=10)
n_virg.visualize(n_back, cutoff=100, magnification=10)
n_vers.visualize(n_back, cutoff=100, magnification=10)

In [ ]:
ratio = 0.5
f_seto = n_seto.create_profile(n_back, ratio=ratio)
f_virg = n_virg.create_profile(n_back, ratio=ratio)
f_vers = n_vers.create_profile(n_back, ratio=ratio)
# The spammiest words are “money,” “systemworks,” “rates,” “sale,” and “year,”
print("top seto: ", "|".join(f_seto[:15]))
print("top virg: ", "|".join(f_virg[:15]))
print("top vers: ", "|".join(f_vers[:15]))

In [ ]:
# print("starting")
cutoff =1
depth = 100

ratio = 0.9
n_standard = n_back
raw_lists = seto + vers + virg
print("Len background lists: ", len(raw_lists))
arr = [[],[],[]]
itr = 0
allhits = []
for node in [n_seto, n_virg, n_vers]:
    node.create_profile(n_standard, ratio=ratio)
    hits = []
    which = []
    for cutoff in range(2,5):
        node.cutoff = cutoff
        hits.append(sum([1 for sentence in raw_lists if node.predict(sentence[:])]))
        which.append("".join(['#' if node.predict(sentence[:]) else '-' for sentence in raw_lists]))
    #print(hits)
    allhits.append(hits)
    #print(which)
    arr[itr] = which
    itr += 1

print(allhits)
with open('res.csv', 'w') as target:
    for idx in range(len(arr[0][0])):
        target.write(" ".join([arr[0][0][idx], arr[1][0][idx], arr[2][0][idx], str(raw_lists[idx]), "\n"]))
! code res.csv

In [ ]:
#%%time
# Populate the main hydra with EEEEEEverything!
hdr = hd.Hydraseq('')
print("total sentences ", len(hydrav))
#[hdr.full_insert(sentence) for sentence in background]
for row in hydrav:
    hdr.insert(" ".join(row))
len(hdr.columns)

In [ ]:
print(hdr.reset().next_nodes)
print()
for n in hdr.reset().next_nodes:
    print(n, n.nexts)

In [ ]:
idx_elem = 10
sentence = " ".join(background[idx_elem])
print(sentence)
print('---------------------------------------------------')

print(hdr.to_hydra_rep(sentence))

def group_to_hydra_rep(group):
    return [hdr.to_hydra_rep(sentence) for sentence in group]

h_background =  group_to_hydra_rep(background)
h_spam =        group_to_hydra_rep(spam)
h_easy_ham =    group_to_hydra_rep(easy_ham)
h_hard_ham =    group_to_hydra_rep(hard_ham)
print(h_spam)

In [ ]:
n_h_background = theseus.Node(h_background)
n_h_spam = theseus.Node(h_spam)
n_h_easy_ham = theseus.Node(h_easy_ham)
n_h_hard_ham = theseus.Node(h_hard_ham)

In [ ]:
cutoff = 1
depth = 100
target = f_spam
theseus.node.count_hits(spam,     target, cutoff, depth)
theseus.node.count_hits(hard_ham, target, cutoff, depth)
theseus.node.count_hits(easy_ham, target, cutoff, depth);

In [ ]:
theseus.node.visualize(n_easy_ham, n_background, axis_lims=(0.0, 1.0), magnification=10.0)

In [ ]:

def get_spiral_dots(node1, node2, ratio):
    x, y, keys = theseus.node.create_xy_table(node1, node2, cutoff1=100, cutoff2=100, ratio=ratio)
    #print(len(x), len(y), len(keys))
    cles = []
    for row in zip(keys, zip(x, y)):
        if row[1][1] != 0:
            #print(row)
            cles.append(row[0])
        else:
            cles.append(row[0])
            #print("bad row ", row)
    return cles

ratio = 0.9
spam_list = get_spiral_dots(n_h_spam,     n_h_background, ratio)
easy_list = get_spiral_dots(n_h_easy_ham, n_h_background, ratio)
hard_list = get_spiral_dots(n_h_hard_ham, n_h_background, ratio)

def new_count_hits(group, lst, thold):
    """group list<list<str>> lst list, thold int"""
    hits = 0
    for sentence in group:
        score = len(set(sentence) & set(lst))
        if score > thold:
            hits += 1
    return hits

print("background", len(h_background))
print("totals ", len(h_spam), len(h_easy_ham), len(h_hard_ham))
thold = 4
target = spam_list
spamers  = new_count_hits(h_spam,     target, thold)
easyhams = new_count_hits(h_easy_ham, target, thold)
hardhams = new_count_hits(h_hard_ham, target, thold)
print("hits   ",spamers, easyhams, hardhams)

In [6]:
from collections import defaultdict
import csv
headers = True
target_col="Species"
features = defaultdict(list)

with open('../tests/data/iris/iris_data.csv', 'r') as fhandle:
    source = csv.DictReader(fhandle) if headers else fhandle
    for line in source:
        features[line[target_col]].append([str(key)+'-'+str(value) for key, value in line.items() if key != target_col])

nodes = [theseus.Node(documents=documents, name=name) for name, documents in features.items()]

nodes